In [1]:
import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def


def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def


def show_graph(graph_def, max_const_size=32):
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

    
def get_input_data(data_file_name, unique_data_file_name):
    data = np.load(data_file_name)
    # 166 bits fingerprint, 1 concentration float, 1 TGI float
    unique_fp = np.load(unique_data_file_name)
    # there is 6252 unique fingerprints and multiple experiments with each

    np.random.shuffle(data)
    test_data, train_data = np.vsplit(data, [100])

    return test_data, train_data, unique_fp

def sample_prior(size=(64, 4)):
    return np.random.normal(size=size)

def batch_gen(data, batch_size=64):
    max_index = data.shape[0]/batch_size

    while True:
        np.random.shuffle(data)
        for i in xrange(max_index):
            yield np.hsplit(data[batch_size*i:batch_size*(i+1)], [-2, -1])

def same_gen(unq_fp, n_examples=64, n_different=1, mu=-5.82, std=1.68):
    '''
    Generator of same fingerprints with different concentraition
    '''
    
    if n_examples % n_different: 
        raise ValueError('n_examples(%s) must be divisible by n_different(%s)' % (n_examples, n_different))
    max_index = unq_fp.shape[0] / n_different
    targets = np.zeros((n_examples, n_examples))
    block_size = n_examples/n_different
    for i in xrange(n_different):
        '''blocks of ones for every block of equal fp's'''
        targets[i*block_size:(i+1)*block_size, i*block_size:(i+1)*block_size] = 1.
    targets = targets > 0
    
    while 1:
        np.random.shuffle(unq_fp)
        for i in xrange(max_index):
            batch_conc = np.random.normal(mu, std, size=(n_examples, 1))
            batch_fp = np.repeat(unq_fp[i*n_different:(i+1)*n_different], [block_size]*n_different, axis=0)
            yield batch_fp, batch_conc, targets

            
def uniform_initializer(size_1, size_2):
    normalized_size = np.sqrt(6) / (np.sqrt(size_1 + size_2))
    return tf.random_uniform([size_1, size_2], minval=-normalized_size, maxval=normalized_size)


def get_collections_from_scope(scope_name):
    return tf.get_collection(tf.GraphKeys.VARIABLES, scope='scope_name')

class AAE(object):
    def __init__(self, 
                 pretrain_batch_size=512, 
                 batch_size=64, 
                 latent_space=4, 
                 input_space=166, 
                 learning_rate=0.01,
                 encoder_layers=2, 
                 decoder_layers=2,
                 discriminator_layers=1):

        self.pretrain_batch_size = pretrain_batch_size
        self.batch_size = batch_size
        self.latent_space = latent_space
        self.learning_rate = learning_rate
        self.encoder_layers = encoder_layers
        self.decoder_layers = decoder_layers
        self.discriminator_layers = discriminator_layers
        self.input_space = input_space

        self.fingerprint_tensor = tf.placeholder(tf.float32, [None, input_space])
        self.prior_tensor = tf.placeholder(tf.float32, [None, latent_space])
        self.conc_tensor = tf.placeholder(tf.float32, [None, 1])
        self.tgi_tensor = tf.placeholder(tf.float32, [None, 1])
        self.targets_tensor = tf.placeholder(tf.bool, [None, None])

        self.visible_tensor = tf.concat(1, [self.fingerprint_tensor, self.conc_tensor])
        self.hidden_tensor = tf.concat(1, [self.prior_tensor, self.tgi_tensor])
        
        # Encoder net: 166+1->128->64->3+1
        
        with tf.name_scope("encoder"):
            encoder = [self.visible_tensor]
            
            sizes = [self.input_space + 1, 128, 64, self.latent_space]

            for layer_number in xrange(encoder_layers):
                with tf.name_scope("encoder-%s" % layer_number):
                    w = tf.Variable(uniform_initializer(sizes[layer_number], sizes[layer_number + 1]), name="W")
                    b = tf.Variable(tf.random_normal([sizes[layer_number + 1]]), name="b")
                    enc_l = tf.nn.relu(tf.add(tf.matmul(encoder[-1], w), b), name="enc_l")
                    encoder.append(enc_l)

            with tf.name_scope("encoder-fp"):
                w = tf.Variable(uniform_initializer(sizes[-2], sizes[-1]), name="W")
                b = tf.Variable(tf.random_normal([sizes[-1]]), name="b")
                self.encoded_fp = tf.add(tf.matmul(encoder[-1], w), b)

        with tf.name_scope("encoder-tgi"):
            w = tf.Variable(uniform_initializer(sizes[-2], 1), name="W")
            b = tf.Variable(tf.random_normal([1]), name="b")
            self.encoded_tgi = tf.add(tf.matmul(encoder[-1], w), b)

        self.encoded = tf.concat(1, [self.encoded_fp, self.encoded_tgi])
        
        # Decoder net: 3+1->64->128->166+1

        sizes = [self.latent_space + 1, 64, 128, self.input_space]

        with tf.name_scope("decoder"):
            decoder = [self.encoded]
            generator = [self.hidden_tensor]

            for layer_number in xrange(decoder_layers):
                with tf.name_scope("decoder-%s" % layer_number):
                    w = tf.Variable(uniform_initializer(sizes[layer_number], sizes[layer_number + 1]), name="W")
                    b = tf.Variable(tf.random_normal([sizes[layer_number + 1]]), name="b")
                    dec_l = tf.nn.tanh(tf.add(tf.matmul(decoder[-1], w), b), name="dec_l")
                    gen_l = tf.nn.tanh(tf.add(tf.matmul(generator[-1], w), b), name="gen_l")
                    decoder.append(dec_l)
                    generator.append(gen_l)

            with tf.name_scope("decoder-fp"):
                w = tf.Variable(uniform_initializer(sizes[-2], sizes[-1]), name="W")
                b = tf.Variable(tf.random_normal([sizes[-1]]), name="b")
                self.decoded_fp = tf.add(tf.matmul(decoder[-1], w), b, name="decoder_fp")
                self.gen_fp = tf.nn.relu(tf.add(tf.matmul(generator[-1], w), b), name="gen_fp")

            with tf.name_scope("decoder-conc"):
                w = tf.Variable(uniform_initializer(sizes[-2], 1), name="W")
                b = tf.Variable(tf.random_normal([1]), name="b")
                self.decoded_conc = tf.add(tf.matmul(decoder[-1], w), b)
                self.gen_conc = tf.add(tf.matmul(generator[-1], w), b)

        # Discriminator net: 3->64->3->1
        with tf.name_scope("discriminator"):
            discriminator_enc = [self.encoded_fp]
            discriminator_prior = [self.prior_tensor]

            sizes = [self.latent_space, 2 * self.latent_space - 2, 1]

            for layer_number in xrange(discriminator_layers):
                with tf.name_scope("discriminator-%s" % layer_number):
                    w = tf.Variable(uniform_initializer(sizes[layer_number], sizes[layer_number + 1]), name="W")
                    b = tf.Variable(tf.random_normal([sizes[layer_number + 1]]), name="b")
                    disc_enc = tf.nn.relu(tf.add(tf.matmul(discriminator_enc[-1], w), b), name="disc_enc")
                    disc_prior = tf.nn.relu(tf.add(tf.matmul(discriminator_prior[-1], w), b), name="disc_prior")
                
                    discriminator_enc.append(disc_enc)
                    discriminator_prior.append(disc_prior)

            with tf.name_scope("discriminator-final"):
                w = tf.Variable(uniform_initializer(sizes[-2], sizes[-1]), name="W")
                b = tf.Variable(tf.random_normal([sizes[-1]]), name="b")
                self.disc_enc = tf.add(tf.matmul(discriminator_enc[-1], w), b, name="disc_enc")
                self.disc_prior = tf.add(tf.matmul(discriminator_prior[-1], w), b, name="disc_prior")

                
        self.disc_loss = tf.reduce_mean(tf.nn.relu(self.disc_prior) - self.disc_prior + tf.log(1.0 + tf.exp(-tf.abs(self.disc_prior)))) + \
            tf.reduce_mean(tf.nn.relu(self.disc_enc) + tf.log(1.0 + tf.exp(-tf.abs(self.disc_enc))))

        fp_norms = tf.sqrt(tf.reduce_sum(tf.square(self.encoded_fp), keep_dims=True, reduction_indices=[1]))
        normalized_fp = tf.div(self.encoded_fp, fp_norms)
        cosines_fp = tf.matmul(normalized_fp, tf.transpose(normalized_fp))
        self.manifold_cost = tf.reduce_mean(1 - tf.boolean_mask(cosines_fp, self.targets_tensor))
            
        self.enc_fp_loss = tf.reduce_mean(tf.nn.relu(self.disc_enc) - self.disc_enc + tf.log(1.0 + tf.exp(-tf.abs(self.disc_enc))))
        self.enc_tgi_loss = tf.reduce_mean(tf.square(tf.sub(self.tgi_tensor, self.encoded_tgi)))
        self.enc_loss = self.enc_fp_loss

        self.dec_fp_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(self.decoded_fp, self.fingerprint_tensor))
        self.dec_conc_loss = tf.reduce_mean(tf.square(tf.sub(self.conc_tensor, self.decoded_conc)))
        self.dec_loss = self.dec_fp_loss + self.dec_conc_loss

        self.train_discriminator = tf.train.AdamOptimizer(self.learning_rate).minimize(self.disc_loss, var_list=tf.get_collection(tf.GraphKeys.VARIABLES, scope='discriminator'))
        self.train_encoder = tf.train.AdamOptimizer(self.learning_rate).minimize(self.enc_loss, var_list=tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder'))
        self.train_manifold = tf.train.AdamOptimizer(self.learning_rate).minimize(self.manifold_cost, var_list=tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder'))
        self.train_reg = tf.train.AdamOptimizer(self.learning_rate).minimize(self.enc_tgi_loss, var_list=tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder') + tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder-tgi'))
        self.train_autoencoder = tf.train.AdamOptimizer(self.learning_rate).minimize(self.dec_loss, var_list=tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder') + tf.get_collection(tf.GraphKeys.VARIABLES, scope='encoder-tgi') + tf.get_collection(tf.GraphKeys.VARIABLES, scope='decoder'))
        
        
        self.test_data, self.train_data, self.unique_fp = get_input_data("mcf7.data.npy", "mcf7.unique.fp.npy")


        
    def train(self):
        init = tf.initialize_all_variables()
        sess = tf.Session()
        saver = tf.train.Saver()

        sames = same_gen(self.unique_fp, n_different=32)

        self.pretrain(self.train_data, sess, init, sames)
        #sess.run(init)
        
        batches = batch_gen(self.train_data, self.batch_size)
        sames = same_gen(self.unique_fp, n_different=32)
        for e in xrange(50):
            if e > 0 and e % 10 == 0:
                saver.save(sess, './uniform.adam.aae.manifold.%de.model.ckpt' % e)
            
            print("epoch #%d" % e)
            
            for u in xrange(10000):
                batch_fp, batch_conc, batch_tgi = batches.next()
                batch_prior = sample_prior()
                sess.run(self.train_discriminator, feed_dict={self.fingerprint_tensor: batch_fp,
                                                         self.conc_tensor: batch_conc,
                                                         self.tgi_tensor: batch_tgi,
                                                         self.prior_tensor: batch_prior})

                batch_fp, batch_conc, batch_tgi = batches.next()

                sess.run(self.train_encoder, feed_dict={self.fingerprint_tensor: batch_fp,
                                                   self.conc_tensor: batch_conc})

        #         same_fp, same_conc, targets = sames.next()
        #         sess.run(train_manifold, feed_dict={fingerprint_tensor: batch_fp,
        #                                             conc_tensor: batch_conc,
        #                                             targets_tensor: targets})
            
                batch_fp, batch_conc, batch_tgi = batches.next()
                sess.run(self.train_reg, feed_dict={self.fingerprint_tensor: batch_fp,
                                               self.conc_tensor: batch_conc,
                                               self.tgi_tensor: batch_tgi})

                batch_fp, batch_conc, batch_tgi = batches.next()
                sess.run(self.train_autoencoder, feed_dict={self.fingerprint_tensor: batch_fp,
                                                       self.conc_tensor: batch_conc,
                                                       self.tgi_tensor: batch_tgi})
                
            else:
                batch_prior = sample_prior((100, self.latent_space))
                losses = sess.run([self.disc_loss, self.enc_fp_loss, self.enc_tgi_loss, self.dec_fp_loss, self.dec_conc_loss],
                                  feed_dict={self.fingerprint_tensor: self.train_data[:, :-2],
                                             self.conc_tensor: self.train_data[:, -2:-1],
                                             self.tgi_tensor: self.train_data[:, -1:],
                                             self.prior_tensor: batch_prior
                                            })
                
                same_fp, same_conc, targets = sames.next()
                m_loss = sess.run(self.manifold_cost, feed_dict={self.fingerprint_tensor: batch_fp,
                                                            self.conc_tensor: batch_conc,
                                                            self.targets_tensor: targets})
                
                discriminator_loss, encoder_fp_loss, encoder_tgi_loss, autoencoder_fp_loss, autoencoder_conc_loss = losses
                print("disc: %f, enc_fp : %f, mani_fp: %f, enc_tgi: %f, dec_fp : %f, dec_conc : %f" % (discriminator_loss/2.,
                                                                                                       encoder_fp_loss,
                                                                                                       m_loss,
                                                                                                       encoder_tgi_loss,
                                                                                                       autoencoder_fp_loss,
                                                                                                       autoencoder_conc_loss))

    def pretrain(self, train_data, sess, init, sames):
        # pretrain generator w/o regressions and decoding

        batches = batch_gen(train_data, self.pretrain_batch_size)
        
        flag = True
        while flag:
            # need to do a few initialization tries, because from some points
            # Generator doesn't converge.
            
            sess.run(init)
            for e in xrange(15):
                print("epoch #%d" % e)
                discriminator_loss = 0.0
                encoder_fp_loss = 0.0
                mani_loss = 0.0
                for u in xrange(1000):
                    batch_fp, batch_conc, _ = batches.next()
                    batch_prior = sample_prior()
                    _, loss = sess.run([self.train_discriminator, self.disc_loss], 
                        feed_dict={self.fingerprint_tensor: batch_fp,
                            self.conc_tensor: batch_conc,
                            self.prior_tensor: batch_prior})
                    discriminator_loss += loss

                    fp_loss = 2.
                    count = 0
                    while fp_loss > 1. and count < 20:
                        batch_fp, batch_conc, _ = batches.next()
                        _, fp_loss = sess.run([self.train_encoder, self.enc_fp_loss], 
                                            feed_dict={self.fingerprint_tensor: batch_fp,
                                            self.conc_tensor: batch_conc,})
                        count += 1
                    else:
                        encoder_fp_loss += fp_loss

                    same_fp, same_conc, targets = sames.next()
                    _, m_loss = sess.run([self.train_manifold, self.manifold_cost], 
                        feed_dict={self.fingerprint_tensor: batch_fp,
                        self.conc_tensor: batch_conc,
                        self.targets_tensor: targets})
                    
                    mani_loss += m_loss

                discriminator_loss /= 1000. * 2.
                encoder_fp_loss /= 1000.
                mani_loss /= 1000.

                print("disc: %f, enc_p: %f, mani: %f" % (discriminator_loss, encoder_fp_loss, mani_loss))
                if (e >= 5) and (encoder_fp_loss < 0.7):
                    flag = False
                    break




In [ ]:
a = AAE()

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
a.train()

epoch #0
disc: 1.716907, enc_p: 0.708333, mani: 0.000020
epoch #1
disc: 0.699811, enc_p: 0.687444, mani: 0.000000
epoch #2
disc: 0.694964, enc_p: 0.691577, mani: 0.000000
epoch #3
disc: 0.693890, enc_p: 0.692508, mani: 0.000000
epoch #4
disc: 0.693481, enc_p: 0.692851, mani: 0.000000
epoch #5
disc: 0.693342, enc_p: 0.692982, mani: 0.000000
epoch #0
disc: 0.694729, enc_fp : 0.693187, mani_fp: 0.216888, enc_tgi: 1781.151245, dec_fp : 0.434717, dec_conc : 0.996942
epoch #1
disc: 0.693147, enc_fp : 0.693199, mani_fp: 0.390732, enc_tgi: 1880.022705, dec_fp : 0.433360, dec_conc : 0.747710
epoch #2
disc: 0.556728, enc_fp : 0.670376, mani_fp: 0.408044, enc_tgi: 2022.729248, dec_fp : 0.439210, dec_conc : 1.245546
epoch #3
disc: 0.693147, enc_fp : 0.693152, mani_fp: 0.056878, enc_tgi: 1703.691162, dec_fp : 0.437781, dec_conc : 1.338224
epoch #4
disc: 0.693147, enc_fp : 0.693134, mani_fp: 0.066458, enc_tgi: 1695.136597, dec_fp : 0.434387, dec_conc : 1.375732
epoch #5
disc: 0.693147, enc_fp : 0.69